# P.O.S tagging using Gated Recurrent Unit (GRU)

`GRU` (Cho et al., 2014) 구조는 간단하게는 이전에 사용하였던 `LSTM`에서 최종 결과물을 산출하는 `gate`를 제외하였다고 이해할 수 있습니다. 그렇기 때문에 `LSTM`과 비슷하게 `vanishing gradient effect`를 방지할 수 있으면서 연산 속도에서도 이득을 볼 수 있는 구조입니다. 이번에는 `GRU`를 이용하여 자동으로 품사 태깅을 수행하는 모델을 만들어 보도록 하겠습니다. 

**Note:** 아래 내용을 실행하기 전, **seq2seq** 구조를 다시 한 번 설명하고 진행하겠습니다. 

**Note:** 전체 코드는 [이 코드](https://github.com/PacktPublishing/Deep-Learning-with-Keras/blob/master/Chapter06/pos_tagging_gru.py)를 토대로 하였으며, 필요에 따라 변경하였습니다.



## Import modules

필요한 모듈들을 불러오겠습니다. 

In [1]:
from keras.layers.core import Activation, Dense, RepeatVector, SpatialDropout1D

from keras.layers.embeddings import Embedding

from keras.layers.recurrent import GRU, LSTM

from keras.layers.wrappers import TimeDistributed, Bidirectional

from keras.models import Sequential

from keras.optimizers import Adam

from keras.preprocessing import sequence

from keras.utils import np_utils

from sklearn.model_selection import train_test_split

import nltk
import collections
import matplotlib.pyplot as plt
import numpy as np
import os

%matplotlib inline
np.random.seed(87)

Using TensorFlow backend.


## Data preparation

학습을 위해서는 품사 정보가 태깅되어 있는 학습 데이터가 필요합니다. `NLTK`를 통해서 Penn Treebank의 10% 샘플을 사용할 수 있습니다. 이 데이터를 사용해서 training 및 test를 진행하겠습니다. 아래 명령어를 사용해서 treebank 데이터를 다운받겠습니다. 

In [2]:
%%bash
python3 -m nltk.downloader -d /usr/local/share/nltk_data treebank punkt

[nltk_data] Downloading package treebank to
[nltk_data]     /usr/local/share/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package punkt to /usr/local/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


`treebank` 데이터는 `nltk` 툴을 이용해서 파싱된 형태로 읽어올 수 있습니다. 다음의 `bash` 명령어를 이용해서 treebank를 저장할 폴더를 만들고, 그 이후의 `python` 코드를 이용해서 treebank의 `문장과 태깅`을 서로 다른 파일에 저장하도록 하겠습니다. 

In [3]:
%%bash
mkdir data/treebank

mkdir: cannot create directory 'data/treebank': File exists


In [4]:
sent_data = open("data/treebank/treebank_sent.txt", "w")
pos_data = open("data/treebank/treebank_pos.txt", "w")

treebank_data = nltk.corpus.treebank.tagged_sents()

for sent in treebank_data:
    words, poss = [], []
    for word, pos in sent:
        if pos == "-NONE-":
            continue
        words.append(word)
        poss.append(pos)
    sent_data.write("{:s}\n".format(" ".join(words)))
    pos_data.write("{:s}\n".format(" ".join(poss)))
sent_data.close()
pos_data.close()

파일이 제대로 작성되었는지 아래 명령어를 통해서 확인해보겠습니다. 

In [5]:
%%bash
head data/treebank/treebank_sent.txt
echo "=================================================================="
head data/treebank/treebank_pos.txt

Pierre Vinken , 61 years old , will join the board as a nonexecutive director Nov. 29 .
Mr. Vinken is chairman of Elsevier N.V. , the Dutch publishing group .
Rudolph Agnew , 55 years old and former chairman of Consolidated Gold Fields PLC , was named a nonexecutive director of this British industrial conglomerate .
A form of asbestos once used to make Kent cigarette filters has caused a high percentage of cancer deaths among a group of workers exposed to it more than 30 years ago , researchers reported .
The asbestos fiber , crocidolite , is unusually resilient once it enters the lungs , with even brief exposures to it causing symptoms that show up decades later , researchers said .
Lorillard Inc. , the unit of New York-based Loews Corp. that makes Kent cigarettes , stopped using crocidolite in its Micronite cigarette filters in 1956 .
Although preliminary findings were reported more than a year ago , the latest results appear in today 's New England Journal of Medicine , a forum like

## Hyperparameters

미리 설정할 수 있는 `hyperparameter`를 미리 정의하도록 하겠습니다. 

In [56]:
sent_file = "data/treebank/treebank_sent.txt"
pos_file = "data/treebank/treebank_pos.txt"

embed_dim = 300  # embedding layer에서 사용할 feature의 수
hidden_dim = 128  # GRU 마지막 결과값이 전달될 fully-connected layer의 노드 갯수
batch_size = 32  # 한 번에 처리할 데이터의 수
num_epoch = 2# 전체 데이터를 반복할 횟수

## Other parameters

이전 `CNN`에서와 마찬가지로 다른 `parameter`를 찾아보도록 하겠습니다. `CNN`에서와 마찬가지로 다음의 정보가 필요합니다. 

 1. 사용된 단어의 수
 2. 가장 긴 문장의 길이
 3. 단어-인덱스 관계를 정의한 `<dict>` 
 4. 인덱스-단어 관계를 정의한 `<dict>`
 5. 품사-인덱스 관계를 정의한 `<dict>` 
 6. 인덱스-품사 관계를 정의한 `<dict>`
 
이전과 다르게 단어만 다루는 것이 아니라 단어의 `품사`도 다루기 때문에 두 개의 `<dict>` 변수가 더 필요하게 되었습니다. 

필요한 정보를 다음의 코드들을 이용해서 찾아보도록 하겠습니다. 두 개의 파일에 대해서 같은 작업을 진행할 예정이므로, 함수를 정의하여서 두 개의 데이터에 대해서 같은 작업을 실행하도록 하겠습니다. 

**Note:** `python`의 `def` 기능을 잘 모르는 경우 간단히 설명하고 진행하겠습니다. 

In [18]:
# Parse sentence

def parse_sentence(filename):
    counter = collections.Counter()
    max_len = 0
    num_sent = 0
    
    with open(filename, "r") as f:
        for line in f:
            words = line.strip().lower().split()

            # get frequency
            for word in words:
                counter[word] += 1

            # get max_len
            if len(words) > max_len:
                max_len = len(words)

            # get num_sent
            num_sent += 1

        f.close()
    return counter, max_len, num_sent

sent_counter, sent_max_len, sent_num_sent = parse_sentence(sent_file)
pos_counter, pos_max_len, pos_num_sent = parse_sentence(pos_file)

print("Sentence Info")
print("\tNumber of words: ", len(sent_counter))
print("\tMaximum length: ", sent_max_len)
print("\tNumber of sents: ", sent_num_sent)
print("=" * 40)
print("POS Info")
print("\tNumber of POS: ", len(pos_counter))
print("\tMaximum length: ", pos_max_len)
print("\tNumber of sents: ", pos_num_sent)        

Sentence Info
	Number of words:  10947
	Maximum length:  249
	Number of sents:  3914
POS Info
	Number of POS:  45
	Maximum length:  249
	Number of sents:  3914


위 코드를 통해 데이터에는 총 10,947개의 고유 단어가 있었고, 고유 품사는 45개 있음을 알 수 있습니다. 가장 긴 문장은 249개이며, 총 데이터 갯수는 3,914개입니다. 고유 단어와 고유 품사의 갯수는 다를 수 있지만, 나머지는 일치하여야 합니다. 

이전까지는 모든 단어를 사용하여 예측해보았기 때문에, 이제는 `<UNK>`를 활용하여서 모델을 구성해보도록 하겠습니다. 따라서 빈도수가 높은 상위 7000개의 단어를 사용하여 주어진 데이터를 학습하고, 품사는 모든 45개의 품사를 예측해보도록 하겠습니다. 가장 긴 문장의 길이는 250으로 설정하도록 하겠습니다. 

In [19]:
seq_len = 250
num_vocab = 7000
num_pos = 45

다른 모델에서와 마찬가지로 단어-인덱스 관계와 품사-인덱스 관계, 그리고 그 역관계를 정의하는 `<dict>`가 필요합니다. 다음의 코드를 사용하여서 각각의 `<dict>`를 정의해보겠습니다. 같은 작업이 두 번 진행되므로 함수를 정의하여 사용하겠습니다. `PAD`와 `UNK`를 정의하기 위해 전체 단어/품사 갯수에 2를 더해주겠습니다.  

In [20]:
def create_dictionary(counter_name, max_num, sentence = False):
    # tmp_word2idx = collections.defaultdict(int)
    if sentence:
        tmp_word2idx = {x[0]: i + 2 for i, x in enumerate(counter_name.most_common(max_num))}
        tmp_word2idx["PAD"] = 0
        tmp_word2idx["UNK"] = 1
    else:
        tmp_word2idx = {x[0]: i + 1 for i, x in enumerate(counter_name.most_common(max_num))}
        tmp_word2idx["PAD"] = 0
    tmp_idx2word = {v: k for k, v in tmp_word2idx.items()}
    
    return tmp_word2idx, tmp_idx2word

sent_word2idx, sent_idx2word = create_dictionary(sent_counter, num_vocab, sentence = True)
pos_word2idx, pos_idx2word = create_dictionary(pos_counter, num_pos)

print(sent_idx2word[0], sent_idx2word[1])
print(pos_idx2word[0])

print(sent_word2idx["government"], sent_idx2word[sent_word2idx["government"]])
print(pos_word2idx["nnp"], pos_idx2word[pos_word2idx["nnp"]])

sent_num_vocab = num_vocab + 2
pos_num_pos = num_pos + 1

PAD UNK
PAD
104 government
3 nnp


함수를 통해 필요한 `<dict>`들을 생성하였으며, 필요한 `PAD`와 `UNK`가 제대로 입력되어있음을 확인할 수 있습니다. 

## Data conversion

이전과 마찬가지로 전체 문장을 `인덱스`의 나열로 변경하여 저장하여야 합니다. `seq2seq`에서 출력은 입력한 **sequence**를 토대로 다른 **sequence**를 출력해 내는 것입니다. 현재 품사 태깅에서는 **250**개의 단어를 입력으로 받아서 **250**개의 품사를 출력하는 `seq2seq` 모델을 만들게 되며, 출력에서 품사는 실제 품사가 아니라 `one-hot encodding`으로 표현이 된 후 `pos_idx2word`를 사용하여 실제 품사로 전환될 것입니다. 

다음의 코드를 이용해서 데이터를 변환하도록 하겠습니다. 이전에는 입력과 출력 데이터를 다루는 방법이 달랐기 때문에 각각 진행을 하였습니다. 지금은 비슷한 과정이 두 번 진행되어야 하기 때문에 함수를 이용해서 진행하도록 하겠습니다. 

In [52]:
def build_tensor(filename, num_sent, word2idx, max_len, category = False):
    data = np.empty((num_sent, ), dtype = list)
    with open(filename, "r") as f:
        sent_num = 0
        for line in f:
            wids = []
            line = line.lower()
            for word in line.strip().split():
                try:
                    wids.append(word2idx[word])
                except KeyError:
                    wids.append(word2idx["UNK"])
            #if category:
            #    data[sent_num] = np_utils.to_categorical(wids, num_classes=len(word2idx))
                # print(line)
                # print(data[sent_num])
            #else:
            data[sent_num] = wids
            sent_num += 1
        f.close()
    
    tensor_data = sequence.pad_sequences(data, maxlen = max_len)
    if category:
        tensor_data = np_utils.to_categorical(tensor_data, num_classes=len(word2idx))
    return tensor_data

X_data = build_tensor(sent_file, sent_num_sent, sent_word2idx, seq_len)
y_data = build_tensor(pos_file, pos_num_sent, pos_word2idx, seq_len, category = True)

print(X_data.shape, y_data.shape)
print(X_data[0])
print(y_data[0][0])

print(sent_word2idx["government"], sent_idx2word[sent_word2idx["government"]])

print("the second to last word of the first sentence: ", sent_idx2word[X_data[0][248]])
print("POS of that word: ", pos_idx2word[np.argmax(y_data[0][248])])

(3914, 250) (3914, 250, 46)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    

최종적으로 아래와 같이 데이터를 `training/test` 데이터로 분할합니다. 전체 데이터 중 20%가 test 데이터로 사용되었습니다. 

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.2, random_state = 42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3131, 250) (783, 250) (3131, 250, 46) (783, 250, 46)


## embedding layer

word2vec과 같은 방법으로 사전 학습된 `embedding layer`를 만들 수도 있지만, 실제로는 word2vec과는 다른 방법으로 품사 태깅은 `embedding layer`가 만들어집니다. 여기에서는 무작위의 값으로 초기화된 `embedding layer`를 만든 이후, 모델의 학습 과정에서 `embedding layer`도 학습하도록 하겠습니다. 

## Build a model

이제는 실제로 `seq2seq` 모델을 작성해보겠습니다. 먼저 입력값은 인덱스로 변환된 *(None x seq_len)* 형태의 데이터입니다. 이 데이터는 **embedding layer**를 거쳐서 *(None x seq_len x 128)* 로 변환이 됩니다(128은 사전에 `hyperparameter`로 설정해둔 **embed_dim**의 값입니다). embedding layer의 결과값은 **GRU** 구조를 통과하게 되고, 그 결과는 *(None x 64)* 의 형태로 나타납니다(64는 사전에 `hyperparameter`로 설정해둔 **hidden_dim**의 값입니다). `encoder-decoder`를 연결하는 **Repeat Vector** 레이어에 GRU 구조의 결과값이 입력되면  *(None x seq_len x 128)* 의 형태를 반환하고 그 결과는 다시 `decoder` 계층의 **GRU**를 역으로 통과하게 됩니다. decoder의 GRU를 통과한 값은 **fully-connected layer**로 연결되어 *(None x seq_len x pos_num_pos)* 의 값을 출력하게 됩니다. 최종 레이어의 값들은 `softmax` 활성화 함수를 사용하여 값이 산출되며, **argmax**의 인덱스를 이용하여 품사 정보를 예측하게 됩니다. 

다음과 같이 모델을 구성해보도록 하겠습니다. 

In [54]:
model = Sequential()
model.add(Embedding(sent_num_vocab, embed_dim, input_length = seq_len))
model.add(SpatialDropout1D(0.2))
model.add(GRU(hidden_dim, dropout = 0.2, recurrent_dropout = 0.2))
model.add(RepeatVector(seq_len))
model.add(GRU(hidden_dim, return_sequences = True))
model.add(TimeDistributed(Dense(pos_num_pos)))
model.add(Activation("softmax"))

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 250, 300)          2100600   
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 250, 300)          0         
_________________________________________________________________
gru_9 (GRU)                  (None, 128)               164736    
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 250, 128)          0         
_________________________________________________________________
gru_10 (GRU)                 (None, 250, 128)          98688     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 250, 46)           5934      
_________________________________________________________________
activation_5 (Activation)    (None, 250, 46)           0         
Total para

## Training and evaluate the model

실제 모델을 학습하도록 하겠습니다. 학습에 사용할 epoch 값은 보통 여러 숫자를 시도해본 이후 `오버피팅(over-fitting)`이 일어나지 않는 수준에서 정하는 것이 일반적입니다. 이번에는 2회 실시한 이후 값을 살펴보겠습니다. 

In [57]:
model.fit(X_train, y_train, batch_size = batch_size, epochs = num_epoch, validation_data = [X_test, y_test])

Train on 3131 samples, validate on 783 samples
Epoch 1/2
3131/3131 [==============================] - 101s 32ms/step - loss: 0.6056 - acc: 0.9043 - val_loss: 0.6030 - val_acc: 0.9019
Epoch 2/2
3131/3131 [==============================] - 91s 29ms/step - loss: 0.5897 - acc: 0.9043 - val_loss: 0.6000 - val_acc: 0.9019


In [58]:
score, acc = model.evaluate(X_test, y_test, batch_size = batch_size)
print("Test score: %.3f, accuracy: %.3f" % (score, acc))

783/783 [==============================] - 5s 7ms/step
Test score: 0.600, accuracy: 0.902


최종적으로 모델은 최종적으로 90%정도 일치하는 출력물을 내놓았다는 것을 확인할 수 있습니다.